In [ ]:
# import torch
# import os
# from models.Text2ProGenModel import Text2AmpModel

# # 1. 基础配置
# # 注意：这里的路径需要替换为你实际环境中的路径
# config = {
#     "lm": "/t9k/mnt/CM/text2amp/weights/pinal-official-t5-large", # T5 模型路径
#     "plm_type": "./weights/progen3-762m",                          # ProGen3 模型路径
# }

# # 检查路径
# if not os.path.exists(config["lm"]) or not os.path.exists(config["plm_type"]):
#     print("Warning: Model paths not found. Please set correct paths in 'config'.")
#     # 如果路径不存在，接下来的代码可能会报错，建议在此处检查环境
# else:
#     # 2. 实例化模型
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = Text2AmpModel(config)
#     model.to(device)
#     model.train() # 设置为训练模式
# #
#     print("\n[Status] Model Architecture constructed successfully.")
#     print(f"[Info] Total layers in ProGen3: {len(model.plm.model.layers)}")
    
#     # 3. 构造虚拟数据 (Dummy Data)
#     # B: Batch size, L_text: 文本长度, L_prot: 蛋白质序列长度
#     B, L_text, L_prot = 2, 10, 20
    
#     # 模拟 T5 Tokenizer 的输出 (文本 ID 和 Mask)
#     text_ids = torch.randint(0, 1000, (B, L_text)).to(device)
#     text_masks = torch.ones((B, L_text)).to(device)
    
#     # 模拟 ProGen3 Tokenizer 的输出 (蛋白质结构 Token ID)
#     structure_token_ids = torch.randint(0, 100, (B, L_prot)).to(device)
    
#     batch = {
#         "text_ids": text_ids,
#         "text_masks": text_masks,
#         "structure_token_ids": structure_token_ids,
#         "labels": structure_token_ids  # 常用作计算 CrossEntropy Loss 的标签
#     }
    
#     # 4. 执行前向传播 (Forward Pass)
#     print("\n[Action] Running forward pass...")
#     outputs = model(batch)
    
#     # 5. 输出结果
#     print("-" * 30)
#     print(f"Loss: {outputs['loss'].item():.4f}")
#     print(f"Logits shape: {outputs['logits'].shape}")
#     print("-" * 30)
    
#     # 6. 验证 Cross-Attention 梯度状态
#     print("\n[Check] Verifying gradient status:")
#     for name, param in model.named_parameters():
#         if "cross_attn" in name:
#             print(f"-> {name}: requires_grad={param.requires_grad}")
#             break # 只打印第一个 cross_attn 示例

In [ ]:
# # 在模型初始化后，开始训练循环前运行这段代码

# # 1. 冻结 T5 Encoder (通常只作为特征提取器)
# for param in model.lm.parameters():
#     param.requires_grad = False

# # 2. 冻结 ProGen3 的原始参数
# # 技巧：我们只让名字里带有 "cross_attn" 的参数保持 requires_grad=True
# for name, param in model.plm.named_parameters():
#     if "cross_attn" in name:
#         param.requires_grad = True
#         print(f"Training: {name}") # 打印一下确认哪些层在训练
#     else:
#         param.requires_grad = False

# # 检查一下可训练参数量
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# all_params = sum(p.numel() for p in model.parameters())
# print(f"\nTrainable params: {trainable_params} / {all_params} ({trainable_params/all_params:.2%})")

In [ ]:
# print(model)

In [ ]:
# import torch

# # 定义保存路径
# save_path = "./weights/text2protein_model/model.pt"
# os.makedirs(os.path.dirname(save_path), exist_ok=True)

# print("正在保存完整模型检查点...")

# # 我们构造一个包含所有必要信息的字典
# # 这样新的类就不需要再依赖原始的 T5/ProGen3 权重路径了
# checkpoint = {
#     # 1. 核心权重
#     "state_dict": model.state_dict(),
    
#     # 2. 两个子模型的配置 (用于构建空骨架)
#     # 注意：我们提取底层的 config 对象
#     "t5_config": model.lm.lm.config, 
#     "progen_config": model.plm.config,
    
#     # 3. 其他辅助信息
#     "lm_dim": model.lm_dim,
#     "model_config": model.config
# }

# torch.save(checkpoint, save_path)
# print(f"模型已成功保存至: {save_path}")
# print("后续请使用新生成的 Text2ProteinGenModel 类加载此文件。")

In [ ]:
# from models.Text2ProteinGenModel import Text2ProteinGenModel

# # 直接加载，不需要传任何 config 字典
# model = Text2ProteinGenModel("./weights/text2protein_model/model.pt")
# model.cuda()

In [ ]:
# import torch
# from transformers import PreTrainedModel

# # 定义保存路径
# save_path = "text2protein_complete.pt"

# print(f"正在打包保存完整模型到 {save_path} ...")

# # 1. 获取 T5 Config 的辅助函数 (自动判断是 model.lm.config 还是 model.lm.lm.config)
# def get_t5_config(model):
#     if hasattr(model.lm, "config"):
#         return model.lm.config
#     elif hasattr(model.lm, "lm") and hasattr(model.lm.lm, "config"):
#         return model.lm.lm.config
#     else:
#         raise ValueError("无法找到 T5 模型的 config，请检查模型结构")

# # 2. 获取 ProGen3 Config 的辅助函数
# def get_progen_config(model):
#     if hasattr(model.plm, "config"):
#         return model.plm.config
#     elif hasattr(model.plm, "model") and hasattr(model.plm.model, "config"):
#         return model.plm.model.config
#     else:
#         # 尝试直接从 model.config 获取（如果你之前的代码有赋值）
#         return model.config

# t5_config = get_t5_config(model)
# progen_config = get_progen_config(model)

# checkpoint = {
#     # 1. 完整的状态字典
#     "state_dict": model.state_dict(),
    
#     # 2. 结构配置
#     "config": {
#         "t5_config": t5_config,
#         "progen_config": progen_config,
#         "lm_dim": model.lm_dim,
#         "vocab_size": progen_config.vocab_size 
#     }
# }

# torch.save(checkpoint, save_path)
# print("保存成功！")
# print(f"T5 Config hidden size: {t5_config.d_model}")
# print(f"ProGen3 Config hidden size: {progen_config.hidden_size}")

In [1]:
import torch
from models.Text2ProteinGenModel import Text2ProteinGenModel

# 1. 指定权重文件路径
checkpoint_path = "text2protein_complete.pt"

print(f"正在从 {checkpoint_path} 加载模型...")

# 2. 实例化并加载 (不需要任何额外的 config 配置，全在 pt 文件里)
model = Text2ProteinGenModel(checkpoint_path)

# 3. 转移到 GPU 并设为评估模式
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("\n=== 模型加载成功！检查信息如下 ===")
print(f"运行设备: {device}")
print(f"T5 文本维度: {model.lm_dim}")
print(f"ProGen3 词表大小: {model.plm.config.vocab_size}")

# 4. 检查一下层结构，确认 CrossAttention 存在
first_layer = model.plm.model.layers[0]
print(f"\n第一层结构类型: {type(first_layer)}")
if hasattr(first_layer, 'cross_attn'):
    print("✅ Cross-Attention 模块已检测到 (注入成功)")
else:
    print("❌ 未检测到 Cross-Attention (结构错误)")

# 5. 简单跑一个 Dummy Forward 验证一切正常
print("\n正在运行测试推理 (Dummy Forward)...")
try:
    dummy_text = torch.randint(0, 100, (1, 10)).to(device)
    dummy_prot = torch.randint(0, 100, (1, 20)).to(device)
    
    batch = {
        "text_ids": dummy_text,
        "text_masks": torch.ones_like(dummy_text),
        "protein_ids": dummy_prot, # 注意这里用了修正后的键名
        "labels": dummy_prot
    }
    
    with torch.no_grad():
        output = model(batch)
    
    print(f"✅ 推理成功！Loss: {output['loss'].item():.4f}")
    print(f"Logits Shape: {output['logits'].shape}")
    
except Exception as e:
    print(f"❌ 推理出错: {e}")

/t9k/mnt/.conda/envs/ref-progen3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/t9k/mnt/.conda/envs/ref-progen3/lib/python3.12/site-packages/flash_attn/ops/triton/layer_norm.py:984: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/t9k/mnt/.conda/envs/ref-progen3/lib/python3.12/site-packages/flash_attn/ops/triton/layer_norm.py:1043: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/t9k/mnt/CM/text2amp/models/Text2ProteinGenModel.py:131: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to

正在从 text2protein_complete.pt 加载模型...
Loading model configuration from text2protein_complete.pt...
Building model architecture...
Loading state dictionary...
Model restored successfully!

=== 模型加载成功！检查信息如下 ===
运行设备: cuda
T5 文本维度: 1024
ProGen3 词表大小: 134

第一层结构类型: <class 'models.Text2ProteinGenModel.ProGen3LayerWithCrossAttn'>
✅ Cross-Attention 模块已检测到 (注入成功)

正在运行测试推理 (Dummy Forward)...
✅ 推理成功！Loss: 16.0000
Logits Shape: torch.Size([1, 20, 134])
